In [3]:
import numpy as np
import pandas as pd
from WindPy import w

# Initialize WindPy
w.start()

# Define date range and alternative assets
start_date = "2020-01-01"
end_date = "2023-12-31"

# Define alternative assets with their respective Wind codes
alternative_assets = {
    "Private Equity": "RUI.GI",  # Russell 2000 Index as a proxy for private equity
    "Venture Capital": "IXIC.GI",  # NASDAQ Composite Index
    "Commodities": "SC.INE",  # INE Crude Oil Futures
    "US Bonds": "G0000891"  # US 10-Year Treasury Bond
}

# Fetch data
def fetch_data(tickers):
    data = {}
    for name, ticker in tickers.items():
        w_data = w.wsd(ticker, "close", start_date, end_date, "Fill=Previous")
        if w_data.ErrorCode != 0:
            raise ValueError(f"Error downloading data for {name} from WindPy")
        dates = w_data.Times
        close_prices = w_data.Data[0]
        df = pd.DataFrame(close_prices, index=dates, columns=[name])
        data[name] = df
    return data

# Get alternative assets data
try:
    alternative_data = fetch_data(alternative_assets)
except ValueError as e:
    print(e)
    w.stop()
    exit()

# Check for empty data
for name, df in alternative_data.items():
    if df.empty:
        print(f"No data retrieved for alternative asset: {name}")

# Merge data
def merge_data(data_dict):
    if not data_dict:
        return pd.DataFrame()
    merged_df = pd.concat(data_dict.values(), axis=1)
    merged_df.dropna(inplace=True)
    return merged_df

# Merge alternative assets data
alternative_df = merge_data(alternative_data)

# Check if merged data is empty
if alternative_df.empty:
    print("Error: Alternative assets Merged DataFrame is empty.")
    w.stop()
    exit()

# Calculate returns
def calculate_returns(df):
    returns = df.pct_change().dropna()
    return returns

alternative_returns = calculate_returns(alternative_df)

# Check if returns data is empty
if alternative_returns.empty:
    print("Error: Alternative assets Returns DataFrame is empty.")
    w.stop()
    exit()

# Calculate correlation matrix
correlation_matrix = alternative_returns.corr()

# Print correlation matrix
print("Correlation Matrix of Alternative Assets:")
print(correlation_matrix.to_string(float_format=lambda x: f"{x:.2f}"))

# Stop WindPy
w.stop()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
Correlation Matrix of Alternative Assets:
                 Private Equity  Venture Capital  Commodities  US Bonds
Private Equity             1.00             0.95         0.09      0.30
Venture Capital            0.95             1.00         0.04      0.20
Commodities                0.09             0.04         1.00      0.10
US Bonds                   0.30             0.20         0.10      1.00
